# Validation de données

## Introduction

## Mise en œuvre

Lorsque l'on utilise la stratégie native de `Laravel` (cf. 'Formulaires'), la validation de données se fait dans le contrôleur.

Admettons que nous recevions un formulaire pour un nouveau produit, dans la classe `ProductContrpller`, via une méthode `store` :

In [ ]:
%%php

class ProductController extends Controller
{
   /**
    * Store a newly created resource in storage.
    *
    * @param  \App\Http\Requests\StoreProductRequest $request
    */
    public function store(StoreProductRequest $request): RedirectResponse
    {
        // Création d'un nouveau produit
        $product = new Product();
        // fusion de l'objet avec les données du formulaire
        $product->fill($request->all());
        // Ecriture dans la base de données
        $procduct->save();
        
        return redirect(...)
        
    }
}

Si nous voulons insérer la procédure de validation, nous devons faire appel à a méthode `validate`de la requête :

In [ ]:
%%php

public function store(StoreProductRequest $request): RedirectResponse
{
    $product = new Product();
    // Validation
    if ($request->validate($rules)) {
        // Suite du code...
    };
}

La variable `rules`contient toutes les contraintes de validité et peut s'écrire sous deux formes :

1. linéaire: 

In [ ]:
%%php

$rules = [
    'title' => 'required|unique:posts|max:255'
];

2 tableau :

In [ ]:
%%php

$rules = [
    'title' => ['required', 'unique:posts', 'max:255']
];

L'ensemble des règles de validité est expliqué dans la [documentation de `Laravel`](https://laravel.com/docs/10.x/validation#available-validation-rules).

### Traitement des erreurs

Si certaines contraintes ne sont pas vérifiées, nous voulons faire trois choses :
- Remplir le formulaire avec des données précedemment entrées
- Renvoyer le formulaire à l'utilisateur
- Afficher les erreurs

#### Remplir à nouveau le formulaire

Nous pouvons avoir recours à la fonction utilitaire `old` qui race les données, qui ont été sauvegardées dans la session.

In [ ]:
%%blade

<input type="text" name="title" value="{{ old('title') }}" />

#### Renvoyer le formulaire

Ici nous pouvons simplement réafficher la vue

In [ ]:
%%php

return view('product.new', [])

#### Afficher les erreurs

Les erreurs peuvent être récupérées via la directive `@errors`

In [ ]:
%%blade

<input type="text" name="title" value="{{ old('title') }}" @error('title') class="is-invalid" @enderror />

@error('title')
    <div class="alert alert-danger">{{ $message }}</div>
@enderror



On peut aussi utiliser une variable globale du nom de `errors`.

## Utiliser des classes de requêtes

Dans la logique de `Laravel`, chaque classe d'entité devrait être associée à une « _classe de requêtes_ ».

C'es ce que nous avons fait dans le premier exemple, et vous pouvez vous reporter au document sur les modèles ([Modèles]())

Cette classe prend en charge la validation au travers de la méthode `rules`. Le corps de cette méthode rend exactement le même tableau que celui que nous avons défini précédemment.

In [ ]:
%%php

class StoreAuthorRequest extends FormRequest
{
    public function rules()
    {
        return [
            'title' => 'required|unique:posts|max:255'];
     }
}

Les classes de requêtes fournissent plusieurs méthodes qui sont très utiles :

1. `fails` : indique quela validation du formulaire a échoué quelque part :

In [ ]:
%%php

if ($request->fails() {
    return view('product.new', ...)
}

2. `validated` : renvoie un tableau de toutes les données validées

In [ ]:
%%php

$safe = $request->validated();

3. `safe` : renvoue un itérateur des données validées, qui peut être filtré avec `only` ou `except` (voire `all`), ce qui marque la différence avec la méthode précedente.

In [ ]:
%%php

$safe = $request->safe()->only(['title', 'price']);